In [43]:
import json

from os import listdir
from os.path import isfile, join, isdir

countries_root = 'json_store/by_country/'

country_folders = [f for f in listdir(countries_root) if not isdir(f)]
print country_folders

#calculates amount of people who immigrated into a country from 
#a certain place within the month 
def immigrated_in_month_from(total_year, total_from_year, total_month):
    percent_from = total_from_year/float(total_year)
    return int(round(total_month*percent_from, 0))
    


# with open('json_store/monthly_by_country.json') as data_file:    
#     monthly_data = json.load(data_file)

['Greece', 'Spain', 'Italy']


In [58]:
all_data = {}

for country_folder in country_folders:
    country_obj = {}
    print country_folder
    with open('{0}{1}/monthly{1}.json'.format(countries_root, country_folder)) as data_file: 
        data_f = json.load(data_file)
        prevYear = data_f[0]["year"] 
        all_months = {}
        monthly = {}
        yearly = {}
        for data in data_f:
            yearly[data["year"]] = yearly.get(data["year"], 0) + data["value"]
            if data["year"] == prevYear:
                all_months[data["month"]] = data["value"]
            else:
                monthly[prevYear] = all_months
                all_months = {}
                all_months[data["month"]] = data["value"]
            prevYear = data["year"]
        monthly[prevYear] = all_months
        all_months[data["month"]] = data["value"]
        country_obj["yearly"] = yearly
        country_obj["monthly"] = monthly
        
    origin_files = [f for f in listdir(countries_root+country_folder) if 'from' in f]
    origins = {}
    for origin_file in origin_files:
        with open('{0}{1}/{2}'.format(countries_root, country_folder, origin_file)) as data_file:
                data = json.load(data_file)[0]
                origins[data["country_of_origin_en"]] = data["value"]
    country_obj["origins"] = origins
    all_data[country_folder] = country_obj

all_data.pop("Malta", None)

print all_data

with open('json_store/all_data.json', 'w') as outfile:
    json.dump(all_data, outfile)

Greece
Spain
Italy
{'Italy': {'yearly': {2016: 56227, 2014: 170100, 2015: 153842}, 'monthly': {2016: {1: 5273, 2: 3828, 3: 9676, 4: 9149, 5: 19925, 6: 8376}, 2014: {1: 2171, 2: 3335, 3: 5459, 4: 15679, 5: 14599, 6: 22641, 7: 24031, 8: 24774, 9: 26107, 10: 15277, 11: 9295, 12: 6732}, 2015: {1: 3528, 2: 4354, 3: 2283, 4: 16063, 5: 21235, 6: 22891, 7: 23186, 8: 22609, 9: 15922, 10: 8916, 11: 3218, 12: 9637}}, 'origins': {u'Ghana': 995, u'Ethiopia': 1012, u'Egypt': 1815, u'Tunisia': 320, u'Senegal': 2565, u'Somalia': 3450, u'Guinea-Bissau': 254, u'Mali': 2614, u'Cameroon': 777, u'Sierra Leone': 217, u'Benin': 183}}, 'Spain': {'yearly': {2016: 1352, 2014: 4348, 2015: 4408}, 'monthly': {2016: {1: 447, 2: 180, 3: 276, 4: 449, 5: 0, 6: 0}, 2014: {1: 144, 2: 33, 3: 232, 4: 148, 5: 325, 6: 246, 7: 264, 8: 1705, 9: 380, 10: 341, 11: 211, 12: 319}, 2015: {1: 241, 2: 44, 3: 267, 4: 240, 5: 438, 6: 377, 7: 346, 8: 385, 9: 459, 10: 795, 11: 508, 12: 308}}, 'origins': {u'Kenya': 4, u'Cameroon': 191, u

In [59]:
processed_data = {}
with open('json_store/all_data.json') as data_file: 
    data = json.load(data_file)
    proc_country_data = {}
    for country_code in country_folders:
        country_data = data[country_code]
        origin_data = country_data["origins"]
        years_data = {}
        for i in range(0, 3):
            year = unicode(i + 2014)
            year_data = {}
            yearsum = 0;
            if "yearly" in country_data:
                for year_key, year_val in country_data["yearly"].iteritems():
                    yearsum += year_val
                months_data = {}
                for month_key, month_val in sorted(country_data["monthly"][year].iteritems()):
                    origin_data = {}
                    for origin, origin_val in country_data["origins"].iteritems():
                        origin_data[str(origin)] = immigrated_in_month_from(yearsum, origin_val, month_val)
                    months_data[int(month_key)] = origin_data
                years_data[int(year)] = months_data
        proc_country_data[country_code] = years_data
processed_data = proc_country_data

with open('json_store/deaths.json') as data_file:            
    processed_data["deaths"] = json.load(data_file)

print processed_data

with open('json_store/processed_data.json', 'w') as outfile:
    json.dump(processed_data, outfile)

{'deaths': {u'2015': {u'11': 106, u'10': 432, u'12': 210, u'1': 82, u'3': 61, u'2': 345, u'5': 95, u'4': 1244, u'7': 230, u'6': 11, u'9': 268, u'8': 686}, u'2014': {u'11': 22, u'10': 126, u'12': 95, u'1': 12, u'3': 10, u'2': 24, u'5': 329, u'4': 50, u'7': 864, u'6': 318, u'9': 813, u'8': 616}, u'2016': {u'11': 0, u'10': 0, u'12': 0, u'1': 369, u'3': 325, u'2': 54, u'5': 1138, u'4': 630, u'7': 0, u'6': 345, u'9': 0, u'8': 0}}, 'Italy': {2016: {1: {'Ghana': 14, 'Tunisia': 4, 'Egypt': 25, 'Ethiopia': 14, 'Senegal': 36, 'Somalia': 48, 'Guinea-Bissau': 4, 'Mali': 36, 'Cameroon': 11, 'Sierra Leone': 3, 'Benin': 3}, 2: {'Ghana': 10, 'Tunisia': 3, 'Egypt': 18, 'Ethiopia': 10, 'Senegal': 26, 'Somalia': 35, 'Guinea-Bissau': 3, 'Mali': 26, 'Cameroon': 8, 'Sierra Leone': 2, 'Benin': 2}, 3: {'Ghana': 25, 'Tunisia': 8, 'Egypt': 46, 'Ethiopia': 26, 'Senegal': 65, 'Somalia': 88, 'Guinea-Bissau': 6, 'Mali': 67, 'Cameroon': 20, 'Sierra Leone': 6, 'Benin': 5}, 4: {'Ghana': 24, 'Tunisia': 8, 'Egypt': 44, 